In [1]:
import matplotlib.backends.backend_pgf
import matplotlib.pyplot as plt
matplotlib.use('module://backend_ipe')

#latex_man = matplotlib.backends.backend_pgf.LatexManager()

import numpy as np
from tensorflow import keras
import multiprocessing as mp
from itertools import repeat

from tfspline import model
from tfspline import plot
from tfspline import sampledata
from tfspline import parallel

## Input data
To simplify choice of constants we shift and scale data such that values are in the [0, 1] range. Otherwise we would need to adapt learning rate and similar things. We skip the back-transformation as we would do in production code.

In [2]:
# Generate input data

data_x = np.linspace(0, 0.5*np.pi, 30)
data_y = np.sin(data_x)

# data_x = np.linspace(0, 1, 100)
# data_y = np.sin(data_x**2 * 4 * np.pi)

[data_x_norm, data_y_norm] = sampledata.rescale_input_data(data_x, data_y)

plt.plot(data_x_norm, data_y_norm, color='gray', linestyle='dotted', linewidth=0.1, marker='.')

In [3]:
learning_rate = 0.1
gradient_regularization=True
deg=5
n=2
continuity=False
seg_overlap = 0
epochs=80
lambd = 1

In [4]:
spline = model.Spline(polydegree=5, polynum=2, ck=2, clamped=True)
spline.continuity = False
spline.verbose = True

opt = keras.optimizers.SGD(learning_rate=learning_rate, momentum=0.95, nesterov=True)
#opt = keras.optimizers.Adam(learning_rate=learning_rate, amsgrad=True)

spline.fit(data_x_norm, data_y_norm, optimizer=opt, n_epochs=epochs, factor_approximation_quality=lambd,
factor_ck_pressure=1-lambd, factor_curvature=0, gradient_regularization=gradient_regularization, overlap_segments=0,
                shift_polynomial_centers='median', uniform_split='True', initialize_l2fit=False)

TensorFlow: Number of recognized GPUs:  1
Fitting took 0.9010028839111328 seconds


<tf.Tensor: shape=(), dtype=float64, numpy=0.0022694210612523953>

In [5]:
plot.plot_spline(spline)
plt.savefig('destination_path.pdf', format='pdf')